In [6]:
import os
import re
import csv
import uuid
import glob
import shutil

In [16]:
# Variables
source_dir = "source_files/"
source_metadata_file = source_dir + "papers.csv"

table_output_dir = "extracted_tables/"
table_code_dir = table_output_dir + "table_code/"
table_result_file = table_output_dir + "tables.csv"

figure_output_dir = "extracted_figures/"
figure_result_file = figure_output_dir + "figures.csv"

possible_extensions = [".pdf", ".png", ".jpg", ".jpeg", ".eps"]

os.makedirs(table_output_dir, exist_ok=True)
os.makedirs(table_code_dir, exist_ok=True)
os.makedirs(figure_output_dir, exist_ok=True)

if os.path.isfile(table_result_file) == False:
    open(table_result_file, "w").close()
    
if os.path.isfile(figure_result_file) == False:
    open(figure_result_file, "w").close()

In [35]:
# Reset directories and csv files
# WARNING: This deletes all collected tables and figures
files = glob.glob(table_code_dir + "*")
for f in files:
    os.remove(f)
    
files = glob.glob(figure_output_dir + "*")
for f in files:
    os.remove(f)
open(figure_result_file, "w").close()

if os.path.isfile(table_result_file):
    os.remove(table_result_file)
    open(table_result_file, "w").close()

PermissionError: [WinError 32] Der Prozess kann nicht auf die Datei zugreifen, da sie von einem anderen Prozess verwendet wird: 'extracted_figures\\figures.csv'

In [33]:
"""
Extracting tables and figures from the downloaded papers in the source directory
Start - index of the first paper to be used (beginning from 0)
End - index of the last paper to be used
"""
def extract_data_from_papers(start, end):
    csvfile_table = open(table_result_file, 'a', newline='', encoding="utf-8")
    csvfile_figure = open(figure_result_file, 'a', newline='', encoding="utf-8")
    csvfile_metadata = open(source_metadata_file, "r")

    spamreader = csv.reader(csvfile_metadata, delimiter=';', quotechar='"', quoting=csv.QUOTE_ALL)

    for index, row in enumerate(spamreader):
        if index < start:
            continue
        if index > end:
            break
        
        paper_path = source_dir + row[0]
        if os.path.isdir(paper_path):
            print("\n\n" + row[0])
            tex_files = [os.path.join(root, file)
                 for root, dirs, files in os.walk(paper_path)
                 for file in files if file.endswith('.tex')]
            print(tex_files)

            complete_tex = ""
            for file in tex_files:
                try:
                    f = open(file, "r", encoding="utf8")
                    complete_tex += f.read()
                    f.close()
                except UnicodeDecodeError as e:
                    print("UnicodeDecodeError occurred. File could not be loaded.")
                    continue

            found_document_header = re.search(r"\\documentclass\[.*?\\begin\{document\}", complete_tex, re.DOTALL)
            if found_document_header is None:
                print("Document header could not be identified.")
                continue

            found_tables = re.findall(r"\\begin\{table\*?\}.*?\\end\{table\*?\}", complete_tex, re.DOTALL)
            found_figures = re.findall(r"\\begin\{figure\*?\}.*?\\end\{figure\*?\}", complete_tex, re.DOTALL)

            for table in found_tables:
                #r"\\caption\{(.*?)}"
                #caption_match = re.search(r"\\caption\{(([^{}]*(\{[^{}]*\})?[^{}]*)+)\}", table)

                """
                label_match = re.search(r"\\label\{(.*?)\}", table)
                if label_match:
                    found_label = label_match.group(1)
                    print(found_label)
                    found_paragraphs = re.findall(fr".*\\ref\{{{re.escape(found_label)}}}.*", complete_tex)
                    for paragraph in found_paragraphs:
                        print(paragraph)
                """

                caption_match = re.search(r"\\caption\{(.*?)}", table)        
                if caption_match:
                    found_caption = caption_match.group(1)
                    table_id = str(uuid.uuid4())

                    # Store meta information in csv file
                    csvfile_table.write(table_id + ";" + row[0] + ";https://arxiv.org/abs/" + row[0] + ";" + found_caption + "\n")

                    # Store as txt file in own data collection
                    table_file_path = table_code_dir + table_id + ".txt"
                    f = open(table_file_path, "w", encoding="utf-8")
                    f.write(table)
                    f.close()

                    # Store as tex file in folder of paper
                    table_file_path = paper_path + "/FR_TAB_" + table_id + ".tex"
                    table_tex_code = found_document_header.group(0) + "\n\pagenumbering{gobble}\n" + table + "\n\end{document}"
                    f = open(table_file_path, "w", encoding="utf-8")
                    f.write(table_tex_code)
                    f.close()

            for figure in found_figures:
                found_graphics = re.findall(r"\\includegraphics(\[.*?\])*\{(.*?)\}", figure)
                #caption_match = re.search(r"\\caption\{(([^{}]*(\{[^{}]*\})?[^{}]*)+)\}", figure)
                caption_match = re.search(r"\\caption\{(.*?)}", figure)
                if caption_match:
                    found_caption = caption_match.group(1)
                    for graphic in found_graphics:
                        figure_id = str(uuid.uuid4())

                        graphic_path = graphic[1]
                        file_type = os.path.splitext(graphic_path)[-1]
                        if os.path.isfile(paper_path+"/"+graphic_path):
                            graphic_path = paper_path+"/"+graphic_path
                        else:
                            for ext in possible_extensions:
                                possible_path = paper_path+"/"+graphic_path + ext
                                if os.path.isfile(possible_path):
                                    graphic_path = possible_path
                                    break

                        try:
                            shutil.copy(graphic_path, figure_output_dir+figure_id+file_type)
                            csvfile_figure.write(figure_id + ";" + row[0] + ";https://arxiv.org/abs/" + row[0] + ";" + found_caption + "\n")
                        except FileNotFoundError as e:
                            print(f"File not found: {graphic_path} - {e}")
                        except Exception as e:
                            print(f"An error occurred: {e}")

            print(f"{len(found_tables)} tables found.")
            print(f"{len(found_figures)} figures found.")

    csvfile_table.close()
    csvfile_figure.close()
    csvfile_metadata.close()

In [34]:
extract_data_from_papers(7,7)



2305.01851
['source_files/2305.01851\\arXiv.tex']
Document header could not be identified.
